In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

c:\Users\admin\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
ori=pd.read_csv('synthetic_road_accidents_100k.csv')

In [3]:
TARGET = 'accident_risk'
BASE = [col for col in train.columns if col not in ['id', TARGET]]
CATS = ['road_type', 'lighting', 'weather', 'road_signs_present', 'public_road', 'time_of_day', 'holiday', 'school_season']
ORIG = []

for col in BASE:
    tmp = ori.groupby(col)[TARGET].mean()
    new_col_name = f"orig_{col}"
    tmp.name = new_col_name
    train = train.merge(tmp, on=col, how='left')
    test = test.merge(tmp, on=col, how='left')
    ORIG.append(new_col_name)

print(len(ORIG), 'Orig Features Created!!')

for df in [train, test, ori]:
    base_risk = (
        0.3 * df["curvature"] + 
        0.2 * (df["lighting"] == "night").astype(int) + 
        0.1 * (df["weather"] != "clear").astype(int) + 
        0.2 * (df["speed_limit"] >= 60).astype(int) + 
        0.1 * (np.array(df["num_reported_accidents"]) > 2).astype(int)
    )
    df['Meta'] = base_risk

train['orig_curvature'] = train['orig_curvature'].fillna(ori[TARGET].mean())
test['orig_curvature'] = test['orig_curvature'].fillna(ori[TARGET].mean())

train[TARGET]-=train['Meta']

12 Orig Features Created!!


In [ ]:
from autogluon.tabular import TabularPredictor

predictor = TabularPredictor(label='accident_risk').fit(
    train_data=train,
    time_limit=3600*3,
    presets='best',
    num_bag_folds=3,
    num_stack_levels=1
)

leaderboard = predictor.leaderboard()
predictions = predictor.predict(test)

ans_df=pd.DataFrame({'id':test['id'],'accident_risk':predictions})
ans_df.to_csv('autogluon_submission.csv',index=False)

No path specified. Models will be saved in: "AutogluonModels\ag-20251031_043525"
Preset alias specified: 'best' maps to 'best_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.10.9
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19041
CPU Count:          20
Memory Avail:       52.64 GB / 63.86 GB (82.4%)
Disk Space Avail:   110.04 GB / 464.50 GB (23.7%)
Presets specified: ['best']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=3, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify 

[1000]	valid_set's rmse: 0.0564856
[2000]	valid_set's rmse: 0.0564701
[1000]	valid_set's rmse: 0.0561237
[2000]	valid_set's rmse: 0.0561089
[1000]	valid_set's rmse: 0.0562026
[2000]	valid_set's rmse: 0.0561772


	-0.0562	 = Validation score   (-root_mean_squared_error)
	20.98s	 = Training   runtime
	2.9s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 1174.66s of the 2674.25s of remaining time.
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=12, gpus=0)


[1000]	valid_set's rmse: 0.056295
[1000]	valid_set's rmse: 0.0559922


	-0.0561	 = Validation score   (-root_mean_squared_error)
	8.64s	 = Training   runtime
	0.93s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 1164.68s of the 2664.27s of remaining time.
	-0.0564	 = Validation score   (-root_mean_squared_error)
	95.89s	 = Training   runtime
	11.29s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 1057.02s of the 2556.61s of remaining time.
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=12, gpus=0)
	-0.0561	 = Validation score   (-root_mean_squared_error)
	654.51s	 = Training   runtime
	0.15s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L1 ... Training model for up to 402.03s of the 1901.62s of remaining time.
	-0.0564	 = Validation score   (-root_mean_squared_error)
	63.75s	 = Training   runtime
	10.16s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ... Training model for up to 327.66s of the 1827.25

...

	-0.0562	 = Validation score   (-root_mean_squared_error)
	23.11s	 = Training   runtime
	3.5s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 3568.57s of the 8065.29s of remaining time.
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=12, gpus=0)
	-0.0561	 = Validation score   (-root_mean_squared_error)
	8.17s	 = Training   runtime
	0.93s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 3559.07s of the 8055.79s of remaining time.
	-0.0564	 = Validation score   (-root_mean_squared_error)
	109.28s	 = Training   runtime
	12.19s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 3437.12s of the 7933.84s of remaining time.
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=12, gpus=0)
	-0.056	 = Validation score   (-root_mean_squared_error)
	728.94s	 = Training   runtime
	0.21s

[1000]	valid_set's rmse: 0.0559718
[2000]	valid_set's rmse: 0.0559483
[3000]	valid_set's rmse: 0.0559476
[1000]	valid_set's rmse: 0.0561311
[2000]	valid_set's rmse: 0.0561132
[3000]	valid_set's rmse: 0.0561131
[1000]	valid_set's rmse: 0.0560552
[2000]	valid_set's rmse: 0.0560337
[3000]	valid_set's rmse: 0.0560363


	-0.056	 = Validation score   (-root_mean_squared_error)
	34.67s	 = Training   runtime
	5.45s	 = Validation runtime
Fitting model: NeuralNetFastAI_r191_BAG_L1 ... Training model for up to 88.63s of the 4585.35s of remaining time.
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=12, gpus=0)
		Import fastai failed. A quick tip is to install via `pip install autogluon.tabular[fastai]==1.4.0`. 
Fitting model: CatBoost_r9_BAG_L1 ... Training model for up to 88.24s of the 4584.96s of remaining time.
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=12, gpus=0)
	Ran out of time, early stopping on iteration 299.
	Ran out of time, early stopping on iteration 327.
	Ran out of time, early stopping on iteration 401.
	-0.0561	 = Validation score   (-root_mean_squared_error)
	80.11s	 = Training   runtime
	0.31s	 = Validation runtime
Fitting model: LightGBM_r96_BAG_L1 ... Training model fo

[1000]	valid_set's rmse: 0.0560853
[2000]	valid_set's rmse: 0.0560779
[3000]	valid_set's rmse: 0.0560767
[1000]	valid_set's rmse: 0.0557151
[2000]	valid_set's rmse: 0.0557103
[1000]	valid_set's rmse: 0.0561383
[2000]	valid_set's rmse: 0.0561282


	-0.056	 = Validation score   (-root_mean_squared_error)
	29.52s	 = Training   runtime
	3.54s	 = Validation runtime
Fitting model: NeuralNetTorch_r22_BAG_L2 ... Training model for up to 233.03s of the 1732.05s of remaining time.
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=12, gpus=0)
	Ran out of time, stopping training early. (Stopping on epoch 7)
	Ran out of time, stopping training early. (Stopping on epoch 7)
	Ran out of time, stopping training early. (Stopping on epoch 10)
	-0.0562	 = Validation score   (-root_mean_squared_error)
	209.52s	 = Training   runtime
	2.85s	 = Validation runtime
Fitting model: XGBoost_r33_BAG_L2 ... Training model for up to 20.15s of the 1519.17s of remaining time.
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=12, gpus=0)
	-0.056	 = Validation score   (-root_mean_squared_error)
	17.16s	 = Training   runtime
	1.68s	 = Validation runtime


In [6]:
leaderboard

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L4,-0.055922,root_mean_squared_error,104.112411,6834.801947,0.007000,3.311000,4,True,40
1,WeightedEnsemble_L3,-0.055925,root_mean_squared_error,76.866006,5376.402946,0.008000,1.418000,3,True,30
2,LightGBM_BAG_L3,-0.055946,root_mean_squared_error,89.776410,6489.670947,0.210003,4.174002,3,True,32
3,CatBoost_BAG_L3,-0.055951,root_mean_squared_error,89.661408,6677.390946,0.095000,191.894001,3,True,34
4,LightGBM_r131_BAG_L2,-0.055955,root_mean_squared_error,41.540005,3560.431949,0.891000,11.175999,2,True,25
5,XGBoost_BAG_L3,-0.055955,root_mean_squared_error,90.776406,6494.231946,1.209998,8.735001,3,True,36
6,CatBoost_r177_BAG_L3,-0.055956,root_mean_squared_error,89.633407,6500.067945,0.066999,14.571001,3,True,39
7,LightGBM_BAG_L2,-0.055956,root_mean_squared_error,40.906004,3553.868950,0.257000,4.613001,2,True,16
8,CatBoost_r9_BAG_L2,-0.055958,root_mean_squared_error,40.893007,3653.736948,0.244003,104.480998,2,True,26
9,CatBoost_BAG_L2,-0.055961,root_mean_squared_error,40.737004,3741.818951,0.087999,192.563001,2,True,18


In [11]:
full=pd.concat([train, test])
n=len(train)
full=pd.get_dummies(full)
full.drop('id', axis=1, inplace=True)
full["num_reported_accidents"] = full["num_reported_accidents"].map({0:0, 1:0, 2:0, 3:2, 4:4, 5:3, 6:1, 7:0})
full

,num_lanes,curvature,speed_limit,road_signs_present,public_road,holiday,school_season,num_reported_accidents,accident_risk,orig_road_type,...,road_type_urban,lighting_daylight,lighting_dim,lighting_night,weather_clear,weather_foggy,weather_rainy,time_of_day_afternoon,time_of_day_evening,time_of_day_morning
0,2,0.06,35,False,True,False,True,0,0.012,0.383005,...,True,True,False,False,False,False,True,True,False,False
1,4,0.99,35,True,False,True,True,0,0.053,0.383005,...,True,True,False,False,True,False,False,False,True,False
2,4,0.63,70,False,True,True,False,0,-0.089,0.382544,...,False,False,True,False,True,False,False,False,False,True
3,4,0.07,35,True,True,False,False,0,0.089,0.382288,...,False,False,True,False,False,False,True,False,False,True
4,1,0.58,60,False,False,True,False,0,0.086,0.382544,...,False,True,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172580,2,0.01,45,False,False,True,True,0,NaN,0.382544,...,False,False,True,False,False,False,True,True,False,False
172581,1,0.74,70,False,True,False,False,0,NaN,0.382544,...,False,True,False,False,False,True,False,True,False,False
172582,2,0.14,70,False,False,True,True,0,NaN,0.383005,...,True,False,True,False,True,False,False,False,True,False
172583,1,0.09,45,True,True,False,True,0,NaN,0.383005,...,True,True,False,False,False,True,False,False,False,True


In [12]:
x = full.drop(['accident_risk'], axis=1)
columns = x.columns
from sklearn.preprocessing import StandardScaler
x = StandardScaler().fit_transform(x)
tx = x[:n]
ty = train['accident_risk'].values
pre = x[n:]

In [13]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from pytabkit import TabM_D_Regressor, RealMLP_TD_Regressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import root_mean_squared_error as rmse
import torch
import optuna
import json

In [14]:
from sklearn.base import BaseEstimator, RegressorMixin
import ydf
ydf.verbose(0)
class YDFmodel(RegressorMixin, BaseEstimator):
    def __init__(self, **kwargs):
        self.model=None
        self.cls=ydf.GradientBoostedTreesLearner(label='accident_risk',task=ydf.Task.REGRESSION, growing_strategy='BEST_FIRST_GLOBAL', **kwargs)

    def fit(self, X, y, target='accident_risk'):
        ds={'feature': X, target: y}
        self.model=self.cls.train(ds)
        return self

    def predict(self, X):
        return self.model.predict({'feature': X})

In [19]:
def xgb_search(trial):
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.6, 1.0, step=0.1)
    reg_alpha = trial.suggest_float('reg_alpha', 1e-6, 0.1, log=True)
    reg_lambda = trial.suggest_float('reg_lambda', 1e-6, 0.1, log=True)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 15, step=1)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 15, step=1)
    n_estimators = trial.suggest_int('n_estimators', 200, 2000, step=100)
    sk = KFold(n_splits = 5, shuffle = True)
    xgb = XGBRegressor(tree_method='hist',
                        colsample_bytree = colsample_bytree, 
                        regreg_alpha = reg_alpha,
                        reg_lambda = reg_lambda,
                        learning_rate = learning_rate, 
                        max_depth = max_depth, 
                        min_child_weight = min_child_weight, 
                        n_estimators = n_estimators)
    val = cross_val_score(xgb, tx, ty, scoring='neg_root_mean_squared_error', cv=sk).mean()
    return -val

xgb_study = optuna.create_study(study_name='xgb', direction="minimize")
xgb_study.optimize(xgb_search, timeout=int(60 * 60), n_trials=300)
xgb_params = xgb_study.best_params

[I 2025-10-22 10:09:22,616] A new study created in memory with name: xgb
[I 2025-10-22 10:10:11,090] Trial 0 finished with value: 0.05651401158318413 and parameters: {'colsample_bytree': 1.0, 'reg_alpha': 0.00642652407464177, 'reg_lambda': 8.474145245186732e-05, 'learning_rate': 0.010069427093588997, 'max_depth': 15, 'min_child_weight': 9, 'n_estimators': 200}. Best is trial 0 with value: 0.05651401158318413.
[I 2025-10-22 10:15:12,726] Trial 1 finished with value: 0.05629095041765271 and parameters: {'colsample_bytree': 0.8, 'reg_alpha': 4.299848502992092e-06, 'reg_lambda': 0.0078019160148970436, 'learning_rate': 0.0014658997248142647, 'max_depth': 14, 'min_child_weight': 11, 'n_estimators': 1700}. Best is trial 1 with value: 0.05629095041765271.
[I 2025-10-22 10:19:03,750] Trial 2 finished with value: 0.05650421520103288 and parameters: {'colsample_bytree': 0.6, 'reg_alpha': 0.0005823937783315564, 'reg_lambda': 0.0022192425766513315, 'learning_rate': 0.004066682029656311, 'max_depth'

In [11]:
def rf_search(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 500, step=100)
    max_depth = trial.suggest_int("max_depth", 3, 15, step=1)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 30, step=2)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 30, step=2)
    sk = KFold(n_splits = 3, shuffle = True)
    rf = RandomForestRegressor(n_estimators=n_estimators,
                                min_samples_split=min_samples_split,
                                min_samples_leaf = min_samples_leaf,
                                max_depth=max_depth,
                                n_jobs = -1)
    val = cross_val_score(rf, tx, ty, scoring='neg_root_mean_squared_error', cv=sk).mean()
    return -val

rf_study = optuna.create_study(study_name='rf')
rf_study.optimize(rf_search, timeout=int(60 * 60), n_trials=300)
rf_params = rf_study.best_params

import json
para = json.dumps(rf_params)
with open('data.txt','a') as f:
    f.write(para)

[I 2025-10-22 12:44:03,265] A new study created in memory with name: rf
[I 2025-10-22 12:45:00,011] Trial 0 finished with value: 0.056182022868712635 and parameters: {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 16, 'min_samples_leaf': 15}. Best is trial 0 with value: 0.056182022868712635.
[I 2025-10-22 12:49:02,765] Trial 1 finished with value: 0.056096734870905086 and parameters: {'n_estimators': 400, 'max_depth': 12, 'min_samples_split': 22, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.056096734870905086.
[I 2025-10-22 12:53:05,869] Trial 2 finished with value: 0.0560948060537669 and parameters: {'n_estimators': 400, 'max_depth': 14, 'min_samples_split': 20, 'min_samples_leaf': 15}. Best is trial 2 with value: 0.0560948060537669.
[I 2025-10-22 12:55:07,430] Trial 3 finished with value: 0.05609833624558482 and parameters: {'n_estimators': 200, 'max_depth': 14, 'min_samples_split': 6, 'min_samples_leaf': 25}. Best is trial 2 with value: 0.0560948060537669.
[I 20

In [12]:
def et_search(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 500, step=100)
    max_depth = trial.suggest_int("max_depth", 3, 15, step=1)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 30, step=2)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 30, step=2)
    sk = KFold(n_splits = 3, shuffle = True)
    et = ExtraTreesRegressor(n_estimators=n_estimators,
                            min_samples_split=min_samples_split,
                            min_samples_leaf = min_samples_leaf,
                            max_depth=max_depth,
                            n_jobs = -1)
    val = cross_val_score(et, tx, ty, scoring='neg_root_mean_squared_error', cv=sk).mean()
    return -val

et_study = optuna.create_study(study_name='et')
et_study.optimize(et_search, timeout=int(60 * 60), n_trials=300)
et_params = et_study.best_params

para = json.dumps(et_params)
with open('data.txt','a') as f:
    f.write(para)

[I 2025-10-22 13:44:39,011] A new study created in memory with name: et
[I 2025-10-22 13:45:59,832] Trial 0 finished with value: 0.056781563630831054 and parameters: {'n_estimators': 200, 'max_depth': 6, 'min_samples_split': 18, 'min_samples_leaf': 29}. Best is trial 0 with value: 0.056781563630831054.
[I 2025-10-22 13:47:20,758] Trial 1 finished with value: 0.05677315649585449 and parameters: {'n_estimators': 200, 'max_depth': 6, 'min_samples_split': 8, 'min_samples_leaf': 17}. Best is trial 1 with value: 0.05677315649585449.
[I 2025-10-22 13:49:59,025] Trial 2 finished with value: 0.056314409846481304 and parameters: {'n_estimators': 300, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 15}. Best is trial 2 with value: 0.056314409846481304.
[I 2025-10-22 13:54:23,775] Trial 3 finished with value: 0.05622518829463458 and parameters: {'n_estimators': 500, 'max_depth': 13, 'min_samples_split': 14, 'min_samples_leaf': 11}. Best is trial 3 with value: 0.05622518829463458.
[I 2

In [20]:
def lgbm_search(trial):
    boosting_type = trial.suggest_categorical("boosting_type", ["gbdt", "goss"])
    n_estimators = trial.suggest_int("n_estimators", 200, 2000, step=100)
    max_depth = trial.suggest_int("max_depth", 3, 15, step=1)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.1, log=True)
    min_child_samples = trial.suggest_int("min_child_samples", 3, 100)
    min_split_gain = trial.suggest_float("min_split_gain", 0.0, 1.0)
    min_child_weight = trial.suggest_float("min_child_weight", 1e-4, 0.01, log=True)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-6, 0.1, log=True),
    reg_lambda = trial.suggest_float("reg_lambda", 1e-6, 0.1, log=True),
    sk = KFold(n_splits = 5, shuffle = True)
    lgbm = LGBMRegressor(boosting_type=boosting_type,
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        max_depth=max_depth,
                        min_child_samples=min_child_samples,
                        min_child_weight=min_child_weight,
                        min_split_gain=min_split_gain,
                        reg_alpha=reg_alpha,
                        reg_lambda=reg_lambda,
                        n_jobs = -1,
                        verbose = -1)
    val = cross_val_score(lgbm, tx, ty, scoring='neg_root_mean_squared_error', cv=sk).mean()
    return -val

lgbm_study = optuna.create_study(study_name='lgbm', direction="minimize")
lgbm_study.optimize(lgbm_search, timeout=int(60 * 60), n_trials=300)
lgbm_params = lgbm_study.best_params

para = json.dumps(lgbm_params)
with open('data.txt','a') as f:
    f.write(para)

[I 2025-10-22 16:58:02,137] A new study created in memory with name: lgbm
[I 2025-10-22 16:58:16,111] Trial 0 finished with value: 0.05695778950349832 and parameters: {'boosting_type': 'gbdt', 'n_estimators': 1700, 'max_depth': 4, 'learning_rate': 0.0879224208535345, 'min_child_samples': 7, 'min_split_gain': 0.1805665218139152, 'min_child_weight': 0.0011357939368421758, 'reg_alpha': 1.0247464270951032e-06, 'reg_lambda': 0.02444954090604944}. Best is trial 0 with value: 0.05695778950349832.
[I 2025-10-22 16:58:25,685] Trial 1 finished with value: 0.0575508301164366 and parameters: {'boosting_type': 'gbdt', 'n_estimators': 1100, 'max_depth': 9, 'learning_rate': 0.041805876002060274, 'min_child_samples': 6, 'min_split_gain': 0.7250477546569779, 'min_child_weight': 0.008921785359106704, 'reg_alpha': 2.44783704010514e-06, 'reg_lambda': 0.00014966563088386357}. Best is trial 0 with value: 0.05695778950349832.
[I 2025-10-22 16:58:37,309] Trial 2 finished with value: 0.057124679640449906 and p

KeyboardInterrupt: 

In [14]:
def hgb_search(trial):
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1, log=True)
    max_iter = trial.suggest_int('max_iter', 200, 2000, step=100)
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 10, 100, step=5)
    max_depth = trial.suggest_int('max_depth', 3, 15, step=1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 10, 100, step=5)
    l2_regularization = trial.suggest_float('l2_regularization', 1e-4, 0.1, log=True)
    sk = KFold(n_splits = 5, shuffle = True)
    hgb = HistGradientBoostingRegressor(learning_rate=learning_rate,
                                         max_iter=max_iter,
                                         max_leaf_nodes=max_leaf_nodes,
                                         max_depth=max_depth,
                                         min_samples_leaf=min_samples_leaf,
                                         l2_regularization=l2_regularization)
    val = cross_val_score(hgb, tx, ty, scoring='neg_root_mean_squared_error', cv=sk).mean()
    return -val

hgb_study = optuna.create_study(study_name='hgb', direction="minimize")
hgb_study.optimize(hgb_search, timeout=int(60 * 60), n_trials=300)
hgb_params = hgb_study.best_params

para = json.dumps(hgb_params)
with open('data.txt','a') as f:
    f.write(para)

[I 2025-10-22 15:45:20,724] A new study created in memory with name: hgb
[I 2025-10-22 15:45:57,922] Trial 0 finished with value: 0.05619898373679956 and parameters: {'learning_rate': 0.016466019809335868, 'max_iter': 1700, 'max_leaf_nodes': 50, 'max_depth': 5, 'min_samples_leaf': 65, 'l2_regularization': 0.06405016658004659}. Best is trial 0 with value: 0.05619898373679956.
[I 2025-10-22 15:46:14,652] Trial 1 finished with value: 0.05609958168809688 and parameters: {'learning_rate': 0.06157484788016917, 'max_iter': 1900, 'max_leaf_nodes': 15, 'max_depth': 9, 'min_samples_leaf': 65, 'l2_regularization': 0.00032510863787057744}. Best is trial 1 with value: 0.05609958168809688.
[I 2025-10-22 15:46:53,338] Trial 2 finished with value: 0.05639118183170788 and parameters: {'learning_rate': 0.012323360831254099, 'max_iter': 900, 'max_leaf_nodes': 70, 'max_depth': 4, 'min_samples_leaf': 20, 'l2_regularization': 0.0002949331674392648}. Best is trial 1 with value: 0.05609958168809688.
[I 2025-1

In [31]:
def cat_search(trial):
    iterations = trial.suggest_int('iterations', 200, 2000, step=100)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1, log=True)
    depth = trial.suggest_int('depth', 3, 15, step=1)
    random_strength = trial.suggest_float('random_strength', 0.5, 1, step=0.1)
    bagging_temperature = trial.suggest_float('bagging_temperature', 0.5, 1, step=0.1)
    border_count = trial.suggest_int('border_count', 5, 100, step=5)
    l2_leaf_reg = trial.suggest_int('l2_leaf_reg', 1, 10, step=1)
    sk = KFold(n_splits = 3, shuffle = True)
    cat = CatBoostRegressor(iterations = iterations,
                                learning_rate = learning_rate,
                                depth = depth,
                                random_strength = random_strength,
                                bagging_temperature = bagging_temperature,
                                border_count = border_count,
                                l2_leaf_reg = l2_leaf_reg,
                                verbose = False, 
                                task_type = 'GPU')
    val = cross_val_score(cat, tx, ty, scoring='neg_root_mean_squared_error', cv=sk).mean()
    return -val

cat_study = optuna.create_study(study_name='cat', direction="minimize")
cat_study.optimize(cat_search, timeout=int(60 * 60), n_trials=100)
cat_params = cat_study.best_params

para = json.dumps(cat_params)
with open('data.txt','a') as f:
    f.write(para)

[I 2025-10-22 17:53:13,289] A new study created in memory with name: cat
[I 2025-10-22 17:53:36,232] Trial 0 finished with value: 0.05639682625390353 and parameters: {'iterations': 1500, 'learning_rate': 0.016197105471995706, 'depth': 5, 'random_strength': 0.8, 'bagging_temperature': 0.8, 'border_count': 10, 'l2_leaf_reg': 8}. Best is trial 0 with value: 0.05639682625390353.
[I 2025-10-22 17:53:50,733] Trial 1 finished with value: 0.05645839782874812 and parameters: {'iterations': 700, 'learning_rate': 0.04628560204286698, 'depth': 3, 'random_strength': 1.0, 'bagging_temperature': 0.8, 'border_count': 70, 'l2_leaf_reg': 2}. Best is trial 0 with value: 0.05639682625390353.
[I 2025-10-22 17:54:25,500] Trial 2 finished with value: 0.05651842457441816 and parameters: {'iterations': 1100, 'learning_rate': 0.011146212490904294, 'depth': 10, 'random_strength': 0.7, 'bagging_temperature': 0.5, 'border_count': 5, 'l2_leaf_reg': 2}. Best is trial 0 with value: 0.05639682625390353.
[I 2025-10-22 

In [ ]:
def ydf_search(trial):
    num_trees = trial.suggest_int('num_trees', 200, 1000, step=100)
    shrinkage = trial.suggest_float('shrinkage', 0.01, 0.1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 15, step=1)
    l2_regularization = trial.suggest_float('l2_regularization', 1e-6, 0.1, log=True)
    l1_regularization = trial.suggest_float('l1_regularization', 1e-6, 0.1, log=True)
    min_examples = trial.suggest_int('min_examples', 1, 20, step=1)
    subsample = trial.suggest_float('subsample', 0.5, 1.0, step=0.1)
    sk = KFold(n_splits=3, shuffle=True)
    ydf = YDFmodel(
        num_trees=num_trees,
        shrinkage=shrinkage,
        max_depth=max_depth,
        l2_regularization=l2_regularization,
        l1_regularization=l1_regularization,
        min_examples=min_examples,
        subsample=subsample,
    )
    val = cross_val_score(ydf, tx, ty, scoring='neg_root_mean_squared_error', cv=sk).mean()
    return -val

ydf_study = optuna.create_study(study_name='ydf', direction="minimize")
ydf_study.optimize(ydf_search, timeout=int(60 * 60), n_trials=300)
ydf_params = ydf_study.best_params

[I 2025-10-25 15:40:11,053] A new study created in memory with name: ydf
[I 2025-10-25 15:41:06,384] Trial 0 finished with value: 0.05604778786859266 and parameters: {'num_trees': 200, 'shrinkage': 0.027139588037625055, 'max_depth': 7, 'l2_regularization': 0.02237640802757754, 'l1_regularization': 4.231119595546045e-06, 'min_examples': 12, 'subsample': 0.5}. Best is trial 0 with value: 0.05604778786859266.
[I 2025-10-25 15:41:58,721] Trial 1 finished with value: 0.056048446397560715 and parameters: {'num_trees': 700, 'shrinkage': 0.04494442372181952, 'max_depth': 12, 'l2_regularization': 7.759523452093936e-05, 'l1_regularization': 0.010976404330934646, 'min_examples': 4, 'subsample': 0.5}. Best is trial 0 with value: 0.05604778786859266.
[I 2025-10-25 15:42:50,714] Trial 2 finished with value: 0.05605358153496448 and parameters: {'num_trees': 1000, 'shrinkage': 0.016893257595808106, 'max_depth': 3, 'l2_regularization': 0.001162864616528192, 'l1_regularization': 2.3239560327211705e-06, 

NameError: name 'json' is not defined

In [31]:
def mlp_search(trial):
    lr = trial.suggest_float('lr', 0.01, 0.1, log=True)
    hidden_sizes = trial.suggest_categorical("hidden_sizes", [[512,256], [256,256], [256,128]])
    p_drop = trial.suggest_float('p_drop', 0.0, 0.2, step=0.05)
    sk = KFold(n_splits=3, shuffle=True, random_state=42)
    mlp = RealMLP_TD_Regressor(
        device='cuda',
        train_metric_name='rmse',
        n_epochs=50,
        batch_size=4096,
        lr=lr,
        hidden_sizes = hidden_sizes,
        p_drop=p_drop,
        use_early_stopping=True,
        early_stopping_additive_patience=5,
        verbosity=0
    )
    val = cross_val_score(mlp, tx, ty, scoring='neg_root_mean_squared_error', cv=sk).mean()
    return -val

mlp_study = optuna.create_study(study_name='mlp', direction="minimize")
mlp_study.optimize(mlp_search, timeout=int(60 * 60), n_trials=100)
mlp_params = mlp_study.best_params

[I 2025-10-25 17:22:30,588] A new study created in memory with name: mlp
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[I 2025-10-25 17:23:33,480] Trial 0 finished with value: 0.05644512517028207 and parameters: {'lr': 0.034076448061572166, 'hidden_sizes': [256, 256], 'p_drop': 0.2}. Best is trial 0 w

In [32]:
def tabm_search(trial):
    lr = trial.suggest_float('lr', 0.01, 0.1, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-2, log=True)
    d_embedding = trial.suggest_int('d_embedding', 16, 64, step=16)
    d_block = trial.suggest_int('d_block', 64, 256, step=64)
    n_blocks = trial.suggest_int('n_blocks', 1, 2)
    dropout = trial.suggest_float('dropout', 0.0, 0.2, step=0.05)
    sk = KFold(n_splits=3, shuffle=True, random_state=42)
    tabm = TabM_D_Regressor(
        device='cuda',
        arch_type='tabm-mini',
        tabm_k=16,
        num_emb_type='none',
        batch_size=4096,
        lr=lr,
        weight_decay=weight_decay,
        n_epochs=50,
        patience=10,
        d_embedding=d_embedding,
        d_block=d_block,
        n_blocks=n_blocks,
        dropout=dropout,
        verbosity=0,
        allow_amp=True,
        gradient_clipping_norm=1.0
    )
    val = cross_val_score(tabm, tx, ty, scoring='neg_root_mean_squared_error', cv=sk).mean()
    return -val

tabm_study = optuna.create_study(study_name='tabm', direction="minimize")
tabm_study.optimize(tabm_search, timeout=int(60 * 60), n_trials=100)
tabm_params = tabm_study.best_params

[I 2025-10-25 18:22:57,473] A new study created in memory with name: tabm


Setting seed: 397307242
Setting seed: 1246035775
Setting seed: 2138792493


[I 2025-10-25 18:23:30,410] Trial 0 finished with value: 0.05682895604377527 and parameters: {'lr': 0.09096418577883127, 'weight_decay': 0.0002468605182893783, 'd_embedding': 16, 'd_block': 192, 'n_blocks': 1, 'dropout': 0.05}. Best is trial 0 with value: 0.05682895604377527.


Setting seed: 389957069
Setting seed: 307324897
Setting seed: 20319763


[I 2025-10-25 18:24:17,048] Trial 1 finished with value: 0.05730693070413989 and parameters: {'lr': 0.05569235915554333, 'weight_decay': 6.930060519262386e-05, 'd_embedding': 48, 'd_block': 192, 'n_blocks': 2, 'dropout': 0.1}. Best is trial 0 with value: 0.05682895604377527.


...

[I 2025-10-25 19:13:47,215] Trial 97 finished with value: 0.056364936864914165 and parameters: {'lr': 0.011094288609512674, 'weight_decay': 8.020816228822835e-05, 'd_embedding': 32, 'd_block': 64, 'n_blocks': 2, 'dropout': 0.0}. Best is trial 72 with value: 0.05635705156900917.


Setting seed: 834653182
Setting seed: 1345748269
Setting seed: 1394381025


[I 2025-10-25 19:14:12,051] Trial 98 finished with value: 0.056458743231271334 and parameters: {'lr': 0.02726871265878686, 'weight_decay': 6.869981536744457e-05, 'd_embedding': 32, 'd_block': 64, 'n_blocks': 2, 'dropout': 0.0}. Best is trial 72 with value: 0.05635705156900917.


Setting seed: 414227069
Setting seed: 125691439
Setting seed: 1634876698


[I 2025-10-25 19:14:40,505] Trial 99 finished with value: 0.056438307151036315 and parameters: {'lr': 0.011134979597340337, 'weight_decay': 6.673830161823208e-05, 'd_embedding': 32, 'd_block': 64, 'n_blocks': 2, 'dropout': 0.05}. Best is trial 72 with value: 0.05635705156900917.


In [34]:
tabm_params

{'lr': 0.010004776454096247,
 'weight_decay': 0.00070929594330862,
 'd_embedding': 32,
 'd_block': 128,
 'n_blocks': 2,
 'dropout': 0.0}

In [35]:
xgb_params = {'tree_method': 'hist',
    'colsample_bytree': 0.6,
    'reg_alpha': 0.0001975454601927612,
    'reg_lambda': 0.07874712733943075,
    'learning_rate': 0.012784793813818576,
    'max_depth': 6,
    'min_child_weight': 6,
    'n_estimators': 2400}
#0.05597195747235208

rf_params = {'n_estimators': 400,
 'max_depth': 13,
 'min_samples_split': 22,
 'min_samples_leaf': 1}

et_params = {'n_estimators': 500,
 'max_depth': 13,
 'min_samples_split': 10,
 'min_samples_leaf': 1}

lgbm_params = {'boosting_type': 'goss',
 'n_estimators': 900,
 'max_depth': 14,
 'learning_rate': 0.030655663980699042,
 'min_child_samples': 16,
 'min_split_gain': 0.0014941548866404279,
 'min_child_weight': 0.00011725285293439913,
 'reg_alpha': 0.002812893265421214,
 'reg_lambda': 0.0001967084920614164}


hgb_params = {'learning_rate': 0.09317673679577433,
 'max_iter': 2000,
 'max_leaf_nodes': 90,
 'max_depth': 13,
 'min_samples_leaf': 60,
 'l2_regularization': 0.013199978618230445}

cat_params = {'iterations': 1400,
 'learning_rate': 0.0587875419978724,
 'depth': 6,
 'random_strength': 0.7,
 'bagging_temperature': 0.7,
 'border_count': 100,
 'l2_leaf_reg': 5}

ydf_params = {'num_trees': 500,
 'shrinkage': 0.0461740048793439,
 'max_depth': 7,
 'l2_regularization': 0.0006140504929943268,
 'l1_regularization': 1.114238274254938e-06,
 'min_examples': 13,
 'subsample': 0.5}

mlp_params = {'lr': 0.0792658212599034, 
              'hidden_sizes': [512, 256], 
              'p_drop': 0.2}

tabm_params = {'lr': 0.010004776454096247,
 'weight_decay': 0.00070929594330862,
 'd_embedding': 32,
 'd_block': 128,
 'n_blocks': 2,
 'dropout': 0.0}

In [37]:
kf=KFold(n_splits=5, shuffle=True)
models=[RandomForestRegressor(**rf_params, n_jobs=-1), 
        ExtraTreesRegressor(**et_params, n_jobs=-1), 
        XGBRegressor(**xgb_params, n_jobs=-1), 
        LGBMRegressor(**lgbm_params, n_jobs=-1, verbose=-1),
        HistGradientBoostingRegressor(**hgb_params),
        CatBoostRegressor(**cat_params, task_type='GPU', verbose=False),
        YDFmodel(**ydf_params),
        RealMLP_TD_Regressor(**mlp_params, n_epochs=50, batch_size=4096, train_metric_name='rmse', device='cuda', use_early_stopping=True, early_stopping_additive_patience=5, verbosity=0),
        TabM_D_Regressor(**tabm_params, arch_type='tabm-mini', batch_size=4096, device='cuda', allow_amp=True, tabm_k=16, num_emb_type='none', n_epochs=50, patience=10, gradient_clipping_norm=1.0)
        ]

ans=[]
for m in models:
    print(m.__class__.__name__)
    oof=[]
    for i,j in kf.split(tx,ty):
        train_x, test_x = tx[i], tx[j]
        train_y, test_y = ty[i], ty[j]
        name=m.__class__.__name__
        m.fit(train_x, train_y)

        print(rmse(test_y, m.predict(test_x)))
        oof.append(m.predict(pre))
    ans.append(np.mean(oof, axis=0))

predictions=np.mean(ans,axis=0)
ans_df=pd.DataFrame({'id':test['id'],'accident_risk':predictions + test['Meta']})
ans_df.to_csv('average_submission.csv',index=False)

RandomForestRegressor
0.05603327512600668
0.05608906933390735
0.05610717657437271
0.055629416989612254
0.0564205682995088
ExtraTreesRegressor
0.056484047373613816
0.05599695261472592
0.05622176242515716
0.05602614997240622
0.05609991047812745
XGBRegressor
0.05585963205688378
0.05605833065875174
0.05569208631196579
0.05619840182153744
0.05611314879952318
LGBMRegressor
0.05603807651108887
0.05622685378567312
0.055675179922509234
0.055814526090805956
0.05616145750609163
HistGradientBoostingRegressor
0.0561132762302824
0.05611780186708648
0.05586425179318247
0.05619427983080314
0.05584455824275527
CatBoostRegressor


0.05630966542960962


0.05573889909282964


0.05571081492452835


0.055712470609482784


...

0.0563807209147746


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


0.056040573458759395


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


0.056206955631956115
TabM_D_Regressor
Setting seed: 1192828837
0.05643390567203319
Setting seed: 236623156
0.05603245885852657
Setting seed: 1342723450
0.05642570248598619
Setting seed: 206956920
0.056453630553386584
Setting seed: 1827621531
0.05642691504358574


In [38]:
from copy import deepcopy
class stacking:
    def __init__(self, cv=5):
        self.estimators=[('rf', RandomForestRegressor(**rf_params, n_jobs=-1)), 
                    ('et', ExtraTreesRegressor(**et_params, n_jobs=-1)), 
                    ('xgb', XGBRegressor(**xgb_params, n_jobs=-1)), 
                    ('lgbm', LGBMRegressor(**lgbm_params, n_jobs=-1, verbose=-1)), 
                    ('cat', CatBoostRegressor(**cat_params, task_type='GPU', verbose=False)),
                    ('hgb', HistGradientBoostingRegressor(**hgb_params)),
                    ('ydf', YDFmodel(**ydf_params)),
                    ('mlp', RealMLP_TD_Regressor(**mlp_params, n_epochs=50, batch_size=4096, train_metric_name='rmse', device='cuda', use_early_stopping=True, early_stopping_additive_patience=5, verbosity=0)),
                    ('tabm', TabM_D_Regressor(**tabm_params, arch_type='tabm-mini', batch_size=4096, device='cuda', allow_amp=True, tabm_k=16, num_emb_type='none', n_epochs=50, patience=10, gradient_clipping_norm=1.0))
                    ]
        self.final_estimator = Ridge(alpha=0.01)
        self.kf=KFold(n_splits=cv, shuffle=True)
        self.trained_estimators = []

    def fit(self, X, y):
        self.trained_estimators = []
        self.meta_features = np.zeros((len(X), len(self.estimators)))

        for est_idx, (name, estimator) in enumerate(self.estimators):
            print(f"Training {name}...")
            fold_models=[]
            for fold, (train_idx, val_idx) in enumerate(self.kf.split(X, y)):
                torch.cuda.empty_cache()
                train_x, val_x = X[train_idx], X[val_idx]
                train_y, val_y = y[train_idx], y[val_idx]

                model = deepcopy(estimator)
                model.fit(train_x, train_y)
                    
                fold_models.append(model)
                val_pred = model.predict(val_x)
                self.meta_features[val_idx, est_idx] = val_pred
            self.trained_estimators.append((name, fold_models))

        self.final_estimator.fit(self.meta_features, y)
            
    def _get_meta_features(self, X):
        meta_features = []
        for name, models in self.trained_estimators:
            oof=[]
            for m in models:
                oof.append(m.predict(X))
            preds = np.mean(oof, axis=0)
            meta_features.append(preds.reshape(-1, 1))
        return np.hstack(meta_features)

    def predict(self, X):
        feature=self._get_meta_features(X)
        return self.final_estimator.predict(feature)

In [39]:
st=stacking()
kf=KFold(n_splits=5, shuffle=True)
oof=[]
for i,j in kf.split(tx,ty):
    train_x, test_x = tx[i], tx[j]
    train_y, test_y = ty[i], ty[j]

    st.fit(train_x, train_y)

    print(rmse(test_y, st.predict(test_x)))
    oof.append(st.predict(pre))

predictions=np.mean(oof,axis=0)
ans_df=pd.DataFrame({'id':test['id'],'accident_risk':predictions + test['Meta']})
ans_df.to_csv('stacking_submission.csv',index=False)

Training rf...
Training et...
Training xgb...
Training lgbm...
Training cat...
Training hgb...
Training ydf...
Training mlp...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU avai

Training tabm...
Setting seed: 2082588750
Setting seed: 1345599827
Setting seed: 132392060
Setting seed: 1680093929
Setting seed: 1394385827


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


0.055940616293690903


...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU avai

Training tabm...
Setting seed: 1440127843
Setting seed: 71195547
Setting seed: 541642462
Setting seed: 113111360
Setting seed: 693512089


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


0.056151627443618575


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [ ]:
import torch
torch.cuda.empty_cache()